In [27]:
import plyvel
import dbobjects_pb2 as KaspadDB
import kbech32 
from tqdm.auto import tqdm

In [28]:
sep = b'/'
level = (0).to_bytes(1, 'little')  # Default level for stores with block-level
ghostdag_data_store = b'block-ghostdag-data'
header_store = b'block-headers'
block_store = b'blocks'
header_count_key = b'block-headers-count'
block_count_key = b'blocks-count'
relations_store = b'block-relations'
candidate_pruning_point_key = b'candidate-pruning-point-hash'
pruning_block_index_key = b'pruning-block-index'
pruning_by_index_store = b'pruning-point-by-index'

In [29]:
db_path = r'D:\kaspad-data\datadir2-cp-23.12T00.30'
db = plyvel.DB(db_path)
prefix = db.get(b'active-prefix')

In [30]:
candidate_bytes = db.get(prefix + sep + candidate_pruning_point_key)
cpp = KaspadDB.DbHash()
cpp.ParseFromString(candidate_bytes)
cpp

hash: "\367\220\226[\377\013D\020\214(\343\037\332q\361#jum!\252\373v\345U\002n\212\3479W\000"

In [31]:
ghostdag_data_bucket = prefix + sep + level + sep + ghostdag_data_store + sep
ghostdag_data_bytes = db.get(ghostdag_data_bucket + cpp.hash)
gdd = KaspadDB.DbBlockGhostdagData()
gdd.ParseFromString(ghostdag_data_bytes)
gdd

blueScore: 2104897
blueWork: "\004\006:\032\331\246\272\275"
selectedParent {
  hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
}
mergeSetBlues {
  hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
}
mergeSetBlues {
  hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
}
bluesAnticoneSizes {
  blueHash {
    hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
  }
  anticoneSize: 1
}
bluesAnticoneSizes {
  blueHash {
    hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
  }
  anticoneSize: 1
}

In [32]:
pp_index_bytes = db.get(prefix + sep + pruning_block_index_key)
pp_index = int.from_bytes(pp_index_bytes, 'little')
pp_bytes = db.get(prefix + sep + pruning_by_index_store + sep + 
                  pp_index.to_bytes(8, 'big'))
pp = KaspadDB.DbHash()
pp.ParseFromString(pp_bytes)
pp

hash: "-2$\203b\3263z\300\301S\207\032-L\275\243r4f\232(\340\256_s\235,\226\225K\366"

In [33]:
# db.close()

In [34]:
blocks_bucket = prefix + sep + block_store + sep
block_bytes = db.get(blocks_bucket + cpp.hash)
b = KaspadDB.DbBlock()
b.ParseFromString(block_bytes)

payload = b.transactions[0].payload
timestamp = b.header.timeInMilliseconds

timestamp, payload, b

(1640023894928,
 b'A\x1e \x00\x00\x00\x00\x00\x00t;\xa4\x0b\x00\x00\x00\x00\x00" \xf8\xce0\x977\xdbo$\xf5\xd7q\xec\xe5\x7f\x1bZ\xabP\x06-3\xf4\x81@s 8o\xb0\xec\xfd\x1f\xac',
 header {
   parents {
     parentHashes {
       hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
     }
     parentHashes {
       hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
     }
   }
   parents {
     parentHashes {
       hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
     }
     parentHashes {
       hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.\304\000j\323/x\376"
     }
   }
   parents {
     parentHashes {
       hash: "\356Y\274\267\371\256\035\375d4L\035\332>\366l\254\213\000\337\236\374G\246zT\377\2010=\3309"
     }
     parentHashes {
       hash: "6\375\001=\017\307\364\306\343X\016\215\"2d\361\362j\005Q\232\357jO.

In [35]:
uint64_len = 8
uint16_len = 2
subsidy_len = uint64_len
pubkey_len_len = 1
pubkey_version_len = uint16_len

pubkey_version = payload[uint64_len + subsidy_len]
pubkey_length = payload[uint64_len + subsidy_len + pubkey_version_len]
pubkey_script = payload[uint64_len + subsidy_len + pubkey_version_len + pubkey_len_len:
                        uint64_len + subsidy_len + pubkey_version_len + pubkey_len_len + pubkey_length]

pubkey_version, pubkey_length, kbech32.toAddress(pubkey_script)

(0, 34, 'kaspa:qruvuvyhxldk7f846ac7eetlrdd2k5qx95elfq2qwvsrsmasan737cn58wk0x')

In [36]:
virtual_utxo_set_key = b'virtual-utxo-set'

for key, value in tqdm(db.iterator(prefix=prefix + sep + virtual_utxo_set_key)):
#     print(key)
#     print(value)
    us = KaspadDB.DbUtxoEntry()
    us.ParseFromString(value)
    print(us)
    break

0it [00:00, ?it/s]

amount: 36738071007
scriptPublicKey {
  script: " \277\022v8z\305\330\036\257\325\216YNO\303e\335hT\222\313\335\235\217Hj\237V\031\202\3326\254"
}
blockDaaScore: 1118034
isCoinbase: true



In [39]:
us.scriptPublicKey.script, us.amount, us.blockDaaScore, us.isCoinbase

(b' \xbf\x12v8z\xc5\xd8\x1e\xaf\xd5\x8eYNO\xc3e\xddhT\x92\xcb\xdd\x9d\x8fHj\x9fV\x19\x82\xda6\xac',
 36738071007,
 1118034,
 True)

In [37]:
# db.close()